# Thermal circuit to state-space

In [1]:
import pandas as pd
import pd_dm4bem

## Assembling and converting to state-space
Consider the disassembled thermal circuits shown in Figure 1 that we want to assemble as shown in Figure 2 and then obtain the state-space representation.

![disassambled_TC](./bldg/ass_TCd.svg)
> Figure 1. Four disassembled thermal circuits: wall_out, TC0, TC1, TC2, TC3.

![disassambled_TC](./bldg/ass_TC.svg)
> Figure 2. The assembling of the four circuits from Figure 1.

The steps to obtain a state-space model from thermal circuits described in a folder are:
1. Obtain the disassambled set of thermal circuits from data given in a folder.

In [2]:
# Disassembled thermal circuits
folder_path = "bldg"
TCd = pd_dm4bem.bldg2TCd(folder_path,
                         TC_auto_number=True)
# For non auto-numbering of thermal circuits TC
# TCd = pd_dm4bem.bldg2TCd(folder_path, TC_auto_number=False)

2. Assemble the set of thermal circuits

In [3]:
# Assembled thermal circuit
# from assembly_matrix.csv')
ass_mat = pd.read_csv(folder_path + '/assembly_matrix.csv')
TCm = pd_dm4bem.assemble_TCd_matrix(TCd, ass_mat)

# from assembly_lists.csv'
ass_lists = pd.read_csv(folder_path + '/assembly_lists.csv')
ass_mat = pd_dm4bem.assemble_lists2matrix(ass_lists)
TCl = pd_dm4bem.assemble_TCd_matrix(TCd, ass_mat)

3. Transform the thermal circuit into state-space representation.

In [4]:
# State-space from TC
[As, Bs, Cs, Ds, us] = pd_dm4bem.tc2ss(TCl)

The [state-space](https://en.m.wikipedia.org/wiki/State-space_representation) is characterized by the matrices __A__ (state), __B__ (input), __C__ (ouput) and __D__ (feedthrough) and by the input vector __u__.

In [5]:
As

,c1_θ0,c2_θ0,ow0_θ1,ow0_θ3
c1_θ0,-0.000240,0.000085,0.000000,0.000002
c2_θ0,0.002857,-0.003925,0.000000,0.000790
ow0_θ1,0.000000,0.000000,-0.000024,0.000002
ow0_θ3,0.000011,0.000107,0.000121,-0.000239


In [6]:
Bs

,c1_q0,c2_q0,c3_q0,ow0_q0,c1_θ0,c2_θ0,ow0_θ0,ow0_θ4
c1_θ0,0.000152,0.000000,0.0,0.000000,9.182736e-07,0.000000,0.000000e+00,8.022399e-08
c2_θ0,0.000000,0.000278,0.0,0.000000,0.000000e+00,0.000031,0.000000e+00,2.600003e-05
ow0_θ1,0.000000,0.000000,0.0,0.000022,0.000000e+00,0.000000,1.970654e-08,0.000000e+00
ow0_θ3,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,2.931595e-07


In [7]:
Cs

,c1_θ0,c2_θ0,ow0_θ1,ow0_θ3
c2_θ0,0.0,1.0,0.0,0.0


In [8]:
Ds

,c1_q0,c2_q0,c3_q0,ow0_q0,c1_θ0,c2_θ0,ow0_θ0,ow0_θ4
c2_θ0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
pd.DataFrame({'u': us})

,u
c1_q0,To
c2_q0,To
c3_q0,Ti_sp
ow0_q0,To
c1_θ0,Φ
c2_θ0,Qa
ow0_θ0,Qo
ow0_θ4,Qi


## Types of notation

The notation used for the nodes and flow can be symbols or numbers (in the case if autonumbering).

The data used in the next examples is extracted from the paper:
> [Ghiaus, C. (2013)](https://hal.science/hal-03605823/document). Causality issue in the heat balance method for calculating the design heating and cooling load. Energy, 50, 292-301.

In [10]:
import pandas as pd
import pd_dm4bem

### Thermal circuit file to state-space

#### File describing the thermal circuit with symbols

Let's consider a thermal circuit having symbols used for numbering the temperature nodes and oriented flow branches (Figure 1). 

![tc_sym](./TC_tc2ss/tc2ss_sym.svg)
> Figure 1. Thermal circuit with symbols.

The thermal circuit from Figure 1 is defined in the file `./TC_tc2ss/TC_tc2ss_sym.csv`. In this case, the nodes and the branches are noted with symbols.

In [11]:
pd.read_csv('./TC_tc2ss/TC_tc2ss_sym.csv')

,A,θso,θw,θsi,θa,G,b
0,qw0,-1,1.0,NaN,NaN,2.9,NaN
1,qw1,NaN,-1.0,1,NaN,2.9,NaN
2,qv,NaN,NaN,NaN,1,38.3,Tov
3,qco,1,NaN,NaN,NaN,250.0,Tow
4,qci,NaN,NaN,-1,1,125.0,NaN
5,C,NaN,4000000.0,NaN,8.2E+04,NaN,NaN
6,f,Qo,NaN,Qi,Qg,NaN,NaN
7,y,NaN,NaN,1,1,NaN,NaN


#### Auto-numbering

The nodes and oriented branches can be auto-numbered. The names of the nodes and branches are composed by the name of the circuit and θ0, θ1, ... , for nodes and q0, q1, ... for branches, e.g., a_θ1.

In [12]:
TC_file = "TC_tc2ss/TC_tc2ss_sym.csv"
TC = pd_dm4bem.file2TC(TC_file, name="a",
                       auto_number=True)
[Asa, Bsa, Csa, Dsa, ua] = pd_dm4bem.tc2ss(TC)
Asa

,a_θ1,a_θ3
a_θ1,-0.000001,7.085614e-07
a_θ3,0.000035,-5.016371e-04


#### Symbols
Alternatively, the nodes and the branches can keep the names given in the `TC` file (the default value of `auto_number`is `False`).

In [13]:
TC_file = "TC_tc2ss/TC_tc2ss_sym.csv"
TC = pd_dm4bem.file2TC(TC_file, name="s")
[Ass, Bss, Css, Dss, us] = pd_dm4bem.tc2ss(TC)
Ass

,s_θw,s_θa
s_θw,-0.000001,7.085614e-07
s_θa,0.000035,-5.016371e-04


Note that the order of symbols corespond to the order given in the description file `./TC_tc2ss/TC_tc2ss_num.csv`not to the alphabetical order.

### File describing the thermal circuit with numbers

Instead of using symbols, the thermal circuit can be numbered (Figure 2).

![tc_num](./TC_tc2ss/tc2ss_num.svg)
> Figure 2. Numbered thermal circuit

The thermal circuit from Figure 1 is defined in the file `./TC_tc2ss/TC_tc2ss_num.csv`. In this case, the nodes and the branches are numbered. 

In [14]:
pd.read_csv('./TC_tc2ss/TC_tc2ss_num.csv')

,A,θ0,θ1,θ2,θ3,G,b
0,q0,NaN,NaN,1,NaN,38.3,Tov
1,q1,1,NaN,NaN,NaN,250.0,Tow
2,q2,-1,NaN,NaN,1.0,2.9,NaN
3,q3,NaN,1,NaN,-1.0,2.9,NaN
4,q4,NaN,-1,1,NaN,125.0,NaN
5,C,NaN,NaN,8.2E+04,4000000.0,NaN,NaN
6,f,Qo,Qi,Qg,NaN,NaN,NaN
7,y,NaN,1,1,NaN,NaN,NaN


In [15]:
TC_file = "TC_tc2ss/TC_tc2ss_num.csv"
TC = pd_dm4bem.file2TC(TC_file, name="n")
[Asn, Bsn, Csn, Dsn, un] = pd_dm4bem.tc2ss(TC)
Asn

,n_θ2,n_θ3
n_θ2,-5.016371e-04,0.000035
n_θ3,7.085614e-07,-0.000001


Note that the values of the state-space representations are the same but the order is different. The order is given by the `.cvs` files that define the thermal circuit.